In [ ]:
!pip install datasets
!pip install peft transformers

In [ ]:
from datasets import load_dataset
import pandas as pd

In [ ]:
# Load train dataset
ds = load_dataset("ailsntua/QEvasion")

# Convert to pandas and keep only useful columns
df_train = ds["train"].to_pandas()[["question","interview_answer",
                                   "label","annotator_id"]]

# Create dictionary with key as the annotator and value the Dataframe with
# only the corresponding sQAs
split_dfs = {
    category: group[["question","interview_answer", "label"]]
    for category, group in df_train.groupby('annotator_id')
    }

In [ ]:
temp_train_set = split_dfs[85]
temp_train_set

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from peft import get_peft_model, LoraConfig, TaskType

# Load pre-trained Roberta model and tokenizer
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,     # Task type: sequence classification
    r=8,                            # Rank of the low-rank adaptation
    lora_alpha=16,                  # Scaling factor
    lora_dropout=0.1,               # Dropout probability for LoRA layers
    target_modules=["query", "value"] # Target modules to inject LoRA into (attention projections)
)

# Add LoRA to the model
model = get_peft_model(model, lora_config)

# Freeze the original model parameters (LoRA will only train the adapters)
for param in model.base_model.parameters():
    param.requires_grad = False

# Define the dataset and data loader
# Assuming the dataset returns a dictionary with 'input_ids', 'attention_mask', and 'labels'
dataset = ...  # Your dataset here
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Move model to the appropriate device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()

# Training loop
for epoch in range(5):  # adjust the number of epochs as needed
    model.train()
    total_loss = 0
    for batch in data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}')

    model.eval()

# Save the model with LoRA adapters
model.save_pretrained("roberta-lora")